In [2]:
:e DataKinds
:e TypeOperators
:e TypeApplications
:e ScopedTypeVariables
:e FlexibleContexts
:e ViewPatterns
:e KindSignatures
:e TypeFamilies
:e RankNTypes
:e GADTs

:s -fplugin GHC.TypeLits.KnownNat.Solver
:s -fplugin GHC.TypeLits.Normalise

import           GHC.TypeLits
import           Numeric.LinearAlgebra.Static
import qualified Numeric.LinearAlgebra.Static as S
import qualified Numeric.LinearAlgebra as LA
import           Data.Proxy
import qualified Numeric.LinearProgramming.L1 as L1
import Data.Maybe
import System.Random hiding (split)
import qualified Data.Vector.Storable as V
import Data.Proxy
import Data.Tuple.HT (fst3)

-- For plotting
import Graphics.Rendering.Chart
import Data.Colour
import Data.Colour.Names
import Data.Default.Class
import Graphics.Rendering.Chart.Backend.Cairo
import Control.Lens
import IHaskellPrelude hiding ((<>))
import Data.List (unfoldr)


In [3]:
-- | Form a block matrix out of component pieces
blockMatrix :: forall m n p q. (KnownNat m, KnownNat n, KnownNat p, KnownNat q) => 
               L m n -> L m p -> L q n -> L q p -> L (m + q) (n + p) 
blockMatrix a b c d = a ||| b 
                        === 
                      c ||| d
 
 
infixl 2 ==^
(==^) :: (KnownNat m, KnownNat n) => R m -> R n -> L (m + n) 1
(==^) x y = col x === col y

infixl 3 ||^
(||^) :: (KnownNat m, KnownNat n) => R m -> R n -> L 1 (m + n)
(||^) x y = row x ||| row y

In [4]:
-- | Here we are solving the problem min. 1/2 x^TQx + c^Tx s.t. Ax = b by using
-- the method of Lagrange multipliers. 
qp_with_lsq_constraints :: forall m n. (KnownNat m, KnownNat n) => 
                           Sym m -> R m -> L n m -> R n -> R m
qp_with_lsq_constraints q c a b = fst . split . uncol $ block <\> y
    where
        block = blockMatrix (unSym q) (tr a) 
                                   a      0
        y =  (-1 * c)
                ==^
                 b